In [170]:
import itertools
import re
import sympy as sp
import numpy as np
from scipy.optimize import minimize
from typing import *

In [171]:
def get_expectation(nr_persons: int, nr_tickets: int) -> sp.core.add.Add:
    """
    Calculate the expected number of tickets bought by a group of people.

    This function takes the number of persons and the number of available tickets as input, and
    computes the expected number of tickets bought by the group. It uses the SymPy library to
    handle symbolic expressions for the probabilities and calculations.

    Args:
        nr_persons (int): The number of persons in the group.
        nr_tickets (int): The number of available tickets.

    Returns:
        sp.core.add.Add: The expected number of tickets bought by the group as a SymPy expression.

    Raises:
        ValueError: If nr_tickets is greater than nr_persons, or if either nr_persons or nr_tickets is
                    equal to or greater than 100.
    """
    if nr_tickets > nr_persons:
        raise ValueError("Nr. persons should be less than nr. tickets.")
    if nr_tickets >= 100 or nr_persons >= 100:
        raise ValueError("Nr. person and nr. tickets should be less than 100.")

    _difference_ = nr_persons - nr_tickets

    ## Creating and ordering sympy variables in table ##
    variable_table = np.empty(shape=(nr_persons,nr_tickets),dtype=object)
    person_indexes = [i+1 for i in range(nr_persons)]
    for ticket in range(nr_tickets):
        for person in range(nr_persons):
            if ticket <= person:
                variable_table[person,ticket] = sp.Symbol("p"+str(person+1)+str(ticket+1))

    ## Calculating all combinations of letting n persons buy k tickets (k<n) ##
    combos = []
    for ticket_number in range(1,nr_tickets+1):
        combs = list(itertools.combinations(person_indexes,ticket_number))
        combos.append([list(comb) for comb in combs])
    combos = np.array(combos,dtype=object)

    ## Calculating expectation value ##
    __term_counter__ = 0
    __expectation = sp.sympify(0)
    for combinations in combos:
        for persons in combinations:
            nr_tickets = len(persons)
            person_nr, ticket_nr = 0, 0
            term, term_weight = sp.sympify(1), sp.sympify(0)
            while person_nr < nr_persons:
                if ticket_nr < nr_tickets:
                    person_buying = persons[ticket_nr]
                    if person_nr + 1 == person_buying:
                        term *= (1-variable_table[person_nr][ticket_nr])   # Buying
                        term_weight += variable_table[person_nr][ticket_nr]
                        ticket_nr += 1
                        if ticket_nr == nr_persons-_difference_:
                            break
                    else:
                        term *= variable_table[person_nr][ticket_nr]      # Not Buying
                else:
                    term *= variable_table[person_nr][ticket_nr]          # Not Buying
                person_nr += 1
            __expectation += term_weight*term
            __term_counter__ += 1
    print("Expectation contains ", __term_counter__, " terms")
    return __expectation

def optimize_equations(symbols: List[sp.Symbol], equations: List[sp.Expr],
                       initial_guess: Union[List[float],np.ndarray[float]]) -> np.ndarray:
    """
    Given a list of symbols, a list of equations, and an initial guess, optimize the values of the symbols
    that minimize the sum of squares of the equations using the scipy.optimize.minimize function.
    Returns an array of optimized values of the symbols.

    Args:
        symbols: A list of sympy symbols to optimize.
        equations: A list of sympy expressions representing the equations to solve.
        initial_guess: A list of initial guesses for the values of the symbols.

    Returns:
        An array of optimized values of the symbols.

    Raises:
        ValueError: If the length of symbols and initial_guess does not match.
    """
    if len(symbols) != len(initial_guess):
        raise ValueError("Length of symbols and initial_guess must match")

    # Create a function that takes in the values of the symbols and returns the equation
    f = sp.lambdify(symbols, equations)

    # Define the optimization problem
    _bounds = [(0, 1) for _var in symbols]
    _problem = minimize(lambda x: np.sum(np.square(f(*x))), initial_guess, bounds=_bounds)

    # Print the optimized values
    for i in range(len(symbols)):
        print(f'{symbols[i]}: {np.round(_problem.x[i], 6)}')

    return _problem.x

# 2 tickets - 3 persons:
##### (Manually):

In [172]:
p11, p21, p31 = sp.symbols("p11, p21, p31")
p22, p32      = sp.symbols("p22, p32")

vars = [p11, p21, p22, p31, p32]
# only 1 of 2 tickets bought amongst 3 persons
term1 = p11*(1-p11)*(1-(1-p22))*(1-(1-p32))
term2 = p21*(1-(1-p11))*(1-p21)*(1-(1-p32))
term3 = p31*(1-(1-p11))*(1-(1-p21))*(1-p31)
# 2 of 2 tickets bought amongst 3 persons
term4 = (p11+p22)*(1-p11)*(1-p22)
term5 = (p11+p32)*(1-p11)*(1-(1-p22))*(1-p32)
term6 = (p21+p32)*(1-(1-p11))*(1-p21)*(1-p32)
expectation = term1+term2+term3+term4+term5+term6

# Writing out diff equations to .txt file for easy Ctrl-C Ctrl-V
file = open("3persons2tickets.txt", "w")
for idx, var in enumerate(vars):
    __ = f"eq{idx+1}="+str(sp.diff(expectation,var))
    file.write(re.sub(r"\s+", "", __)+"\n\n")
file.close()

equations = [sp.diff(expectation, var) for var in vars]
initial_guess = [0.5 for i in range(len(vars))]
_ = optimize_equations(vars, equations, initial_guess)

table = np.zeros(shape=(3,2))
for idx, var in enumerate(vars):
    col, row = int(str(var)[-1])-1, int(str(var)[-2])-1
    table[row][col] = np.round(_[idx],5)
print("\n == Corresponding to: == ")
print(table)

p11: 0.554688
p21: 0.5
p22: 0.625
p31: 0.5
p32: 0.5

 == Corresponding to: == 
[[0.55469 0.     ]
 [0.5     0.625  ]
 [0.5     0.5    ]]


##### (Automatically):

In [173]:
expectation = get_expectation(nr_persons=3,nr_tickets=2)
equations = [sp.diff(expectation, var) for var in vars]
initial_guess = [0.5 for i in range(len(vars))]
_ = optimize_equations(vars, equations, initial_guess)

table = np.zeros(shape=(3,2))
for idx, var in enumerate(vars):
    col, row = int(str(var)[-1])-1, int(str(var)[-2])-1
    table[row][col] = np.round(_[idx],5)
print("\n == Corresponding to: == ")
print(table)

Expectation contains  6  terms
p11: 0.554688
p21: 0.5
p22: 0.625
p31: 0.5
p32: 0.5

 == Corresponding to: == 
[[0.55469 0.     ]
 [0.5     0.625  ]
 [0.5     0.5    ]]


# 2 tickets - 4 persons:

In [174]:
p11, p21, p31, p41 = sp.symbols('p11, p21, p31, p41')
p22, p32, p42      = sp.symbols('p22, p32, p42')

vars = [p11, p21, p31, p41, p22, p32, p42]

# Only 1 of 3 tickets bought amongst 4 persons
term1 = p11*(1-p11)*(1-(1-p22))*(1-(1-p32))*(1-(1-p42))
term2 = p21*(1-(1-p11))*(1-p21)*(1-(1-p32))*(1-(1-p42))
term3 = p31*(1-(1-p11))*(1-(1-p21))*(1-p31)*(1-(1-p42))
term4 = p41*(1-(1-p11))*(1-(1-p21))*(1-(1-p31))*(1-p41)

# 2 of 2 tickets bought amongst 4 persons
term5 = (p11+p22)*(1-p11)*(1-p22)
term6 = (p11+p32)*(1-p11)*(1-(1-p22))*(1-p32)
term7 = (p11+p42)*(1-p11)*(1-(1-p22))*(1-(1-p32))*(1-p42)
term8 = (p21+p32)*(1-(1-p11))*(1-p21)*(1-p32)
term9 = (p21+p42)*(1-(1-p11))*(1-p21)*(1-(1-p32))*(1-p42)
term10 = (p31+p42)*(1-(1-p11))*(1-(1-p21))*(1-p31)*(1-p42)

# Expectation
expectation = term1 + term2 + term3 + term4 + \
              term5 + term6 + term7 + term8 + \
              term9 + term10

# Writing out diff equations to .txt file for easy Ctrl-C Ctrl-V
file = open("4persons2tickets.txt", "w")
for idx, var in enumerate(vars):
    __ = f"eq{idx+1}="+str(sp.diff(expectation,var))
    file.write(re.sub(r"\s+", "", __)+"\n\n")
file.close()

equations = [sp.diff(expectation, var) for var in vars]
initial_guess = [0.5 for i in range(len(vars))]
_ = optimize_equations(vars, equations, initial_guess)
table = np.zeros(shape=(4,2))
for idx, var in enumerate(vars):
    col, row = int(str(var)[-1])-1, int(str(var)[-2])-1
    table[row][col] = np.round(_[idx],5)
print("\n == Corresponding to: == ")
print(table)

p11: 0.607422
p21: 0.554686
p31: 0.5
p41: 0.5
p22: 0.695312
p32: 0.625
p42: 0.5

 == Corresponding to: == 
[[0.60742 0.     ]
 [0.55469 0.69531]
 [0.5     0.625  ]
 [0.5     0.5    ]]


###### (Automatically):

In [175]:
expectation = get_expectation(nr_persons=4,nr_tickets=2)
equations = [sp.diff(expectation, var) for var in vars]
initial_guess = [0.5 for i in range(len(vars))]
_ = optimize_equations(vars, equations, initial_guess)

table = np.zeros(shape=(4,2))
for idx, var in enumerate(vars):
    col, row = int(str(var)[-1])-1, int(str(var)[-2])-1
    table[row][col] = np.round(_[idx],5)
print("\n == Corresponding to: == ")
print(table)

Expectation contains  10  terms
p11: 0.607422
p21: 0.554686
p31: 0.5
p41: 0.5
p22: 0.695312
p32: 0.625
p42: 0.5

 == Corresponding to: == 
[[0.60742 0.     ]
 [0.55469 0.69531]
 [0.5     0.625  ]
 [0.5     0.5    ]]


# 3 Tickets - 4 persons:

In [176]:
p11, p21, p31, p41 = sp.symbols('p11, p21, p31, p41')
p22, p32, p42      = sp.symbols('p22, p32, p42')
p33, p43           = sp.symbols('p33, p43')

vars = [p11, p21, p31, p41, p22, p32, p42, p33, p43]

# 1 of 3 tickets bought amongst 4 persons
term1 = p11*(1-p11)*(1-(1-p22))*(1-(1-p32))*(1-(1-p42))
term2 = p21*(1-(1-p11))*(1-p21)*(1-(1-p32))*(1-(1-p42))
term3 = p31*(1-(1-p11))*(1-(1-p21))*(1-p31)*(1-(1-p42))
term4 = p41*(1-(1-p11))*(1-(1-p21))*(1-(1-p31))*(1-p41)

# 2 of 3 tickets bought amongst 4 persons
term5 = (p11+p22)*(1-p11)*(1-p22)*(1-(1-p33))*(1-(1-p43))
term6 = (p11+p32)*(1-p11)*(1-(1-p22))*(1-p32)*(1-(1-p43))
term7 = (p11+p42)*(1-p11)*(1-(1-p22))*(1-(1-p32))*(1-p42)
term8 = (p21+p32)*(1-(1-p11))*(1-p21)*(1-p32)*(1-(1-p43))
term9 = (p21+p42)*(1-(1-p11))*(1-p21)*(1-(1-p32))*(1-p42)
term10 = (p31+p42)*(1-(1-p11))*(1-(1-p21))*(1-p31)*(1-p42)

# 3 of 3 tickets bought amongst 4 persons
term11 = (p11+p22+p33)*(1-p11)*(1-p22)*(1-p33)
term12 = (p11+p22+p43)*(1-p11)*(1-p22)*(1-(1-p33))*(1-p43)
term13 = (p11+p32+p43)*(1-p11)*(1-(1-p22))*(1-p32)*(1-p43)
term14 = (p21+p32+p43)*(1-(1-p11))*(1-p21)*(1-p32)*(1-p43)

# Expectation
expectation = term1 + term2 + term3 + term4 + \
              term5 + term6 + term7 + term8 + \
              term9 + term10 + term11 + term12 + \
              term13 + term14

# Writing out diff equations to .txt file for easy Ctrl-C Ctrl-V
file = open("4persons3tickets.txt", "w")
for idx, var in enumerate(vars):
    __ = f"eq{idx+1}="+str(sp.diff(expectation,var))
    file.write(re.sub(r"\s+", "", __)+"\n\n")
file.close()

equations = [sp.diff(expectation, var) for var in vars]
initial_guess = [0.5 for i in range(len(vars))]
_ = optimize_equations(vars, equations, initial_guess)
table = np.zeros(shape=(4,3))
for idx, var in enumerate(vars):
    col, row = int(str(var)[-1])-1, int(str(var)[-2])-1
    table[row][col] = np.round(_[idx],5)
print("\n == Corresponding to: == ")
print(table)

p11: 0.525848
p21: 0.5
p31: 0.5
p41: 0.5
p22: 0.554691
p32: 0.5
p42: 0.5
p33: 0.625006
p43: 0.5

 == Corresponding to: == 
[[0.52585 0.      0.     ]
 [0.5     0.55469 0.     ]
 [0.5     0.5     0.62501]
 [0.5     0.5     0.5    ]]


##### (Automatically):

In [177]:
expectation = get_expectation(nr_persons=4,nr_tickets=3)
equations = [sp.diff(expectation, var) for var in vars]
initial_guess = [0.5 for i in range(len(vars))]
_ = optimize_equations(vars, equations, initial_guess)

table = np.zeros(shape=(4,3))
for idx, var in enumerate(vars):
    col, row = int(str(var)[-1])-1, int(str(var)[-2])-1
    table[row][col] = np.round(_[idx],5)
print("\n == Corresponding to: == ")
print(table)

Expectation contains  14  terms
p11: 0.525848
p21: 0.5
p31: 0.5
p41: 0.5
p22: 0.554691
p32: 0.5
p42: 0.5
p33: 0.625006
p43: 0.5

 == Corresponding to: == 
[[0.52585 0.      0.     ]
 [0.5     0.55469 0.     ]
 [0.5     0.5     0.62501]
 [0.5     0.5     0.5    ]]


# 2 tickets - 5 persons
##### (Automatically):


In [178]:
p11, p21, p31, p41, p51 = sp.symbols('p11, p21, p31, p41, p51')
p22, p32, p42, p52      = sp.symbols('p22, p32, p42, p52')

vars = [p11, p21, p31, p41, p51, p22, p32, p42, p52]

expectation = get_expectation(nr_persons=5,nr_tickets=2)
equations = [sp.diff(expectation, var) for var in vars]
initial_guess = [0.5 for i in range(len(vars))]
_ = optimize_equations(vars, equations, initial_guess)

table = np.zeros(shape=(5,2))
for idx, var in enumerate(vars):
    col, row = int(str(var)[-1])-1, int(str(var)[-2])-1
    table[row][col] = np.round(_[idx],5)
print("\n == Corresponding to: == ")
print(table)

Expectation contains  15  terms
p11: 0.651129
p21: 0.607422
p31: 0.554688
p41: 0.5
p51: 0.5
p22: 0.741737
p32: 0.695311
p42: 0.625002
p52: 0.5

 == Corresponding to: == 
[[0.65113 0.     ]
 [0.60742 0.74174]
 [0.55469 0.69531]
 [0.5     0.625  ]
 [0.5     0.5    ]]


# 3 tickets - 5 persons:
##### (Automatically):

In [179]:
## Creating sympy variable ##
person_indexes = [i+1 for i in range(5)]
vars_string = ""
for ticket in range(3):
    for person in range(5):
        if ticket <= person:
            vars_string += "p"+str(person+1)+str(ticket+1)+", "
vars_string = vars_string[:-2]
vars = sp.symbols(vars_string)

expectation = get_expectation(nr_persons=5,nr_tickets=3)
equations = [sp.diff(expectation, var) for var in vars]
initial_guess = [0.5 for i in range(len(vars))]
_ = optimize_equations(vars, equations, initial_guess)

table = np.zeros(shape=(5,3))
for idx, var in enumerate(vars):
    col, row = int(str(var)[-1])-1, int(str(var)[-2])-1
    table[row][col] = np.round(_[idx],5)
print("\n == Corresponding to: == ")
print(table)

Expectation contains  25  terms
p11: 0.5612
p21: 0.525849
p31: 0.5
p41: 0.5
p51: 0.5
p22: 0.60742
p32: 0.554688
p42: 0.5
p52: 0.5
p33: 0.69532
p43: 0.624999
p53: 0.5

 == Corresponding to: == 
[[0.5612  0.      0.     ]
 [0.52585 0.60742 0.     ]
 [0.5     0.55469 0.69532]
 [0.5     0.5     0.625  ]
 [0.5     0.5     0.5    ]]


# Testing higher numbers
##### (Automatically):

In [180]:
nr_persons = 9
nr_tickets = 4

person_indexes = [i+1 for i in range(nr_persons)]
vars_string = ""
for ticket in range(nr_tickets):
    for person in range(nr_persons):
        if ticket <= person:
            vars_string += "p"+str(person+1)+str(ticket+1)+", "
vars_string = vars_string[:-2]
vars = sp.symbols(vars_string)

print("Computing expectation...")
expectation = get_expectation(nr_persons=nr_persons,nr_tickets=nr_tickets)
print("Computing derivatives...")
equations = [sp.diff(expectation, var) for var in vars]
initial_guess = [0.5 for i in range(len(vars))]
print("Computing optimization...")
_ = optimize_equations(vars, equations, initial_guess)

table = np.zeros(shape=(nr_persons,nr_tickets))
for idx, var in enumerate(vars):
    col, row = int(str(var)[-1])-1, int(str(var)[-2])-1
    table[row][col] = np.round(_[idx],5)
print("\n == Corresponding to: == ")
print(table)

Computing expectation...
Expectation contains  255  terms
Computing derivatives...
Computing optimization...
p11: 0.614743
p21: 0.588954
p31: 0.561805
p41: 0.53508
p51: 0.512476
p61: 0.5
p71: 0.5
p81: 0.5
p91: 0.5
p22: 0.657384
p32: 0.628901
p42: 0.596639
p52: 0.561198
p62: 0.525848
p72: 0.5
p82: 0.5
p92: 0.5
p33: 0.715864
p43: 0.68669
p53: 0.651128
p63: 0.607422
p73: 0.554687
p83: 0.5
p93: 0.5
p44: 0.800414
p54: 0.775101
p64: 0.741696
p74: 0.695309
p84: 0.624999
p94: 0.5

 == Corresponding to: == 
[[0.61474 0.      0.      0.     ]
 [0.58895 0.65738 0.      0.     ]
 [0.56181 0.6289  0.71586 0.     ]
 [0.53508 0.59664 0.68669 0.80041]
 [0.51248 0.5612  0.65113 0.7751 ]
 [0.5     0.52585 0.60742 0.7417 ]
 [0.5     0.5     0.55469 0.69531]
 [0.5     0.5     0.5     0.625  ]
 [0.5     0.5     0.5     0.5    ]]


First, let's define the terms:

- E[n, k] represents the expected revenue with n potential customers and k available tickets.

- p[n, k] is the optimal price for the k-th ticket when there are n potential customers.

- v[n, k] is the value of the k-th ticket for the n-th potential customer.

The first equation calculates the expected revenue E[n, k]:

- If the value of the k-th ticket for the n-th customer, v[n, k], is greater than or equal to the optimal price p[n, k], the customer will buy the ticket. The probability of this happening is Pr[v[n, k] ≥ p[n, k]] = 1 - p[n, k]. In this case, the revenue generated is the price of the ticket, p[n, k], plus the expected revenue from selling the remaining k - 1 tickets to the n - 1 remaining customers, E[n - 1, k - 1].

 - If the value of the k-th ticket for the n-th customer, v[n, k], is less than the optimal price p[n, k], the customer will not buy the ticket. The probability of this happening is Pr[v[n, k] < p[n, k]] = p[n, k]. In this case, the revenue generated is the expected revenue from selling the remaining k tickets to the n - 1 remaining customers, E[n - 1, k].

Now trying:
\begin{align}
\text{E}[n,k]&=\text{Pr}[v_{n,k}\geq p_{n,k}](p_{n,k}+\text{E}[n-1,k-1])+\text{Pr}[v_{n,k}<p_{n,k}]\text{E}[n-1,k] \\
&=(1-p_{n,k})(p_{n,k}+\text{E}[n-1,k-1])+(1-(1-p_{n,k}))\text{E}[n-1,k] \\
&=(1-p_{n,k})(p_{n,k}+\text{E}[n-1,k-1])+p_{n,k}\text{E}[n-1,k]
\end{align}
s.t.
\begin{align}
\frac{\partial}{\partial p_{n,k}}\Big(\text{E}[n,k]\Big)&=1-2p_{n,k}-\text{E}[n-1,k-1]+\text{E}[n-1,k] \\
\end{align}
solving =0:
\begin{equation}
p_{n,k}=\frac{1-\text{E}[n-1,k-1]+\text{E}[n-1,k]}{2}
\end{equation}

In [184]:
def p_kn(n,k,exps):
    return (1-exps[n-1,k-1]+exps[n-1,k])/2

def e_kn(n,k,exps):
    return (1-p_kn(n,k,exps))*(p_kn(n,k,exps)+exps[n-1,k-1])+p_kn(n,k,exps)*exps[n-1,k]


n,k = 9, 4
# Formatted as (n,k): value
P = np.zeros(shape=(n+1,k+1),dtype=float)
E = np.zeros(shape=(n+1,k+1),dtype=float)
# Edge cases
P[(1, 1)] = 1/2
E[(1,1)] =  1/4
for _n in range(0,n+1):
    for _k in range(0,k+1):
        if _k == 0 or _n == 0:
            E[_n,_k] = 0
        else:
            P[_n, _k] = p_kn(_n,_k,E)
            E[_n, _k] = e_kn(_n,_k,E)

A = np.flip(P[1:,1:])
print(A)
print(table)

In [185]:
A = np.flip(P[1:,1:])
A

array([[0.61474022, 0.68229473, 0.76047871, 0.84982141],
       [0.58895429, 0.65738255, 0.7400724 , 0.83644654],
       [0.56180777, 0.62889437, 0.71584682, 0.8203006 ],
       [0.53510137, 0.5966169 , 0.68669026, 0.80037567],
       [0.51259012, 0.56119947, 0.6511289 , 0.7750815 ],
       [0.5       , 0.52584839, 0.60742188, 0.74172974],
       [0.5       , 0.5       , 0.5546875 , 0.6953125 ],
       [0.5       , 0.5       , 0.5       , 0.625     ],
       [0.5       , 0.5       , 0.5       , 0.5       ]])

In [183]:
table

array([[0.61474, 0.     , 0.     , 0.     ],
       [0.58895, 0.65738, 0.     , 0.     ],
       [0.56181, 0.6289 , 0.71586, 0.     ],
       [0.53508, 0.59664, 0.68669, 0.80041],
       [0.51248, 0.5612 , 0.65113, 0.7751 ],
       [0.5    , 0.52585, 0.60742, 0.7417 ],
       [0.5    , 0.5    , 0.55469, 0.69531],
       [0.5    , 0.5    , 0.5    , 0.625  ],
       [0.5    , 0.5    , 0.5    , 0.5    ]])